In [1]:
import os
# import plotly.graph_objects as go
from tqdm import tqdm
from gym import envs
import argparse
import numpy as np
from bigmdp.data.buffer import StandardBuffer,ReplayBuffer, gather_data_in_buffer, get_iter_indexes
from bigmdp.mdp.MDP_GPU import FullMDP
from bigmdp.mdp.MDP_GPU_bkp import FullMDP as FullMDP_bkp

from bigmdp.utils.utils_eval import evaluate_on_env
from bigmdp.mdp.agent import SimpleAgent
from bigmdp.mdp.agent_bkp import SimpleAgent as SimpleAgent_bkp
from copy import deepcopy as cpy
from os import path
from arg_def import * 
import gym
from sklearn.neighbors import KDTree
from IPython import display
import torch
import time
import copy 
from collections import defaultdict
import heapq
from bigmdp.mdp.MDP_GPU import init2zero, init2zero_def_dict,init2zero_def_def_dict

In [2]:
parser, ArgumentDict = get_argument_parser()
options = "--env_name CartPole-v0 --action_repeat 1 --max_episode_length 200 --MAX_S_COUNT 12000 --MAX_NS_COUNT 5 --mdp_build_k 5 --normalize_by_distance"
args = parser.parse_args(options.split(" "))


In [3]:
from image_wrappers import *
env = gym.make(args.env_name)

In [4]:
train_buffer = ReplayBuffer(state_dim = env.observation_space.shape[0],
                           is_atari= False, 
                           atari_preprocessing= None, 
                           batch_size=32, 
                           buffer_size=20000,
                           device="cpu")

In [5]:
train_buffer, info = gather_data_in_buffer(train_buffer, env,policy = lambda s:np.random.randint(2), episode_count=99999, frame_count=10000)

Average Reward of collected trajectories:22.781


In [6]:
class DummyNet():
    def __init__(self, sim, add_noise=False):
        self.simulator = sim
        self.use_cuda = False
#         self.add_noise = add_noise

    def encode_single(self, o):
        return tuple(o)

    def encode_batch(self, o_batch):
        return [tuple(o) for o in o_batch]

    def predict_single_transition(self, o, a):
        return False

    def predict_batch_transition(self, o_batch, a_batch):
        return False


In [7]:

empty_MDP = FullMDP(A= list(range(env.action_space.n)),
                    ur=args.unknown_transition_reward,
                    vi_params={"gamma":  args.gamma,
                               "slip_prob": args.slip_probability,
                               "rmax_reward": args.rmax_reward,
                               "rmax_thres": args.rmax_threshold,
                               "balanced_explr": args.balanced_exploration,
                              "rmin":-1000},
                    mdp_params={"weight_transitions": args.weight_transitions,
                            "mdp_build_k":args.mdp_build_k,
                            "plcy_lift_k": 1,
                            "knn_delta" : args.knn_delta,
                            "calc_action_vector":False,
                            "penalty_beta":args.penalty_beta,
                            "penalize_unknown_transitions": args.penalty_type != "none"
                            },
                    MAX_S_COUNT= args.MAX_S_COUNT,
                    MAX_NS_COUNT= args.MAX_NS_COUNT,
                    default_mode=args.def_device)

myAgent =  SimpleAgent(empty_MDP, net = DummyNet(None),  fill_with = args.fill_with,
                       sample_neighbors = False, pred_error_threshold = args.within_radius,
                       penalty_type=args.penalty_type, penalty_beta = args.penalty_beta,
                       abstraction_flag=False, normalize_by_distance= args.normalize_by_distance)

mdp_id = "L[{}]-E[{}]-B[{}]-Fill[{}_K{}]-Penalty[{}_{}]".format(args.latent_type, args.encoder_type, args.bottleneck_size, 
                                                          args.fill_with, args.mdp_build_k, args.penalty_type, args.penalty_beta)

myAgent.build_mdp(train_buffer)
# build_mdp(myAgent,train_buffer)


 20%|█▉        | 1954/10000 [00:00<00:00, 19531.17it/s]

Step 1 (Parse Transitions):  Running
Step 1 [Parse Transitions]:  Complete,  Time Elapsed: 0.0509951114654541


Step 2 [Seed Seen Transitions + Unknown (s,a) pairs]:  Running
Len of to commit transitions 10000
ABstraction Faldg False


  2%|▏         | 384/20002 [00:00<00:05, 3833.20it/s]

Len of to commit sa pairs 20002
Step 2 (Commit Seen Transitions):  Complete,  Time Elapsed: 0.6031379699707031 


Step 3 [Commit all Transitions]:  Running


100%|██████████| 20002/20002 [00:05<00:00, 3833.99it/s]


Step 3 (Commit UnSeen Transitions):  Complete,  Time Elapsed: 5.219402551651001
Step 4 [Solve MDP]:  Running
Elapsed Time:1s, VI Error:0.08259583, #Backups: 250
Elapsed Time:2s, VI Error:0.0067215, #Backups: 500
Elapsed Time:2s, VI Error:0.00058746, #Backups: 750
Time takedn to solve 2.744408369064331
% of missing transitions 0.0
Step 4 [Solve MDP]:  Complete,  Time Elapsed: 3.8254377841949463




In [8]:
# empty_MDP_bkp.tranidxMatrix_cpu = empty_MDP.tranidxMatrix_cpu 
# empty_MDP_bkp.tranProbMatrix_cpu = empty_MDP.tranProbMatrix_cpu 
# empty_MDP_bkp.rewardCountMatrix_cpu = empty_MDP.rewardCountMatrix_cpu 
# empty_MDP_bkp.rewardMatrix_cpu = empty_MDP.rewardMatrix_cpu 


In [9]:
# empty_MDP_bkp.solve()

In [10]:
# empty_MDP_bkp.valueDict

In [11]:

empty_MDP_bkp = FullMDP_bkp(A= list(range(env.action_space.n)),
                    ur=args.unknown_transition_reward,
                    vi_params={"gamma":  args.gamma,
                               "slip_prob": args.slip_probability,
                               "rmax_reward": args.rmax_reward,
                               "rmax_thres": args.rmax_threshold,
                               "balanced_explr": args.balanced_exploration,
                              "rmin":-1000},
                    mdp_params={"weight_transitions": args.weight_transitions,
                            "mdp_build_k":args.mdp_build_k,
                            "plcy_lift_k": 1,
                            "knn_delta" : args.knn_delta,
                            "calc_action_vector":False,
                            "penalty_beta":args.penalty_beta,
                            "penalize_unknown_transitions": args.penalty_type != "none"
                            },
                    MAX_S_COUNT= args.MAX_S_COUNT,
                    MAX_NS_COUNT= args.MAX_NS_COUNT,
                    default_mode=args.def_device)

myAgent_bkp =  SimpleAgent_bkp(empty_MDP_bkp, net = DummyNet(None),  fill_with = args.fill_with,
                       sample_neighbors = False, pred_error_threshold = args.within_radius,
                       penalty_type=args.penalty_type, penalty_beta = args.penalty_beta,
                       abstraction_flag=False, normalize_by_distance= args.normalize_by_distance)

mdp_id = "L[{}]-E[{}]-B[{}]-Fill[{}_K{}]-Penalty[{}_{}]".format(args.latent_type, args.encoder_type, args.bottleneck_size, 
                                                          args.fill_with, args.mdp_build_k, args.penalty_type, args.penalty_beta)

myAgent_bkp.build_mdp(train_buffer)
# build_mdp(myAgent,train_buffer)


  7%|▋         | 721/10000 [00:00<00:01, 7203.53it/s]

Step 1 (Parse Transitions):  Running
Step 1 [Parse Transitions]:  Complete,  Time Elapsed: 0.0613255500793457


Step 2 [Seed Seen Transitions + Unknown (s,a) pairs]:  Running
Len of to commit transitions 10000
ABstraction Faldg False


  2%|▏         | 327/20002 [00:00<00:06, 3265.28it/s]

Len of to commit sa pairs 20002
Step 2 (Commit Seen Transitions):  Complete,  Time Elapsed: 1.5845236778259277 


Step 3 [Commit all Transitions]:  Running


100%|██████████| 20002/20002 [00:06<00:00, 3301.30it/s]


Step 3 (Commit UnSeen Transitions):  Complete,  Time Elapsed: 6.06307315826416
Step 4 [Solve MDP]:  Running
Elapsed Time:0s, VI Error:0.0825882, #Backups: 250
Elapsed Time:1s, VI Error:0.0067215, #Backups: 500
Elapsed Time:1s, VI Error:0.00059509, #Backups: 750
Time takedn to solve 1.825568675994873
% of missing transitions 0.0
Step 4 [Solve MDP]:  Complete,  Time Elapsed: 2.1918575763702393




In [13]:
# empty_MDP.valueDict

In [14]:
# empty_MDP_bkp.valueDict

In [78]:
empty_MDP_bkp.tranidxMatrix_cpu = empty_MDP.tranidxMatrix_cpu.astype("float32")
empty_MDP_bkp.tranProbMatrix_cpu = empty_MDP.tranProbMatrix_cpu 
# empty_MDP_bkp.rewardCountMatrix_cpu = empty_MDP.rewardCountMatrix_cpu 
empty_MDP_bkp.rewardMatrix_cpu = empty_MDP.rewardMatrix_cpu 


In [79]:
# for s in empty_MDP.tD:
#     for a in empty_MDP.tD[s]:
#         print(empty_MDP.s2i[s],a)
#         ns_tuples = [(empty_MDP.s2i[ns],round(empty_MDP.tD[s][a][ns],2), round(empty_MDP.rD[s][a][ns],2)) for ns in empty_MDP.tD[s][a]]
#         print("\t", ns_tuples)
#         ns_tuples = [(empty_MDP_bkp.s2idx[ns],round(empty_MDP_bkp.tD[s][a][ns],2), round(empty_MDP_bkp.rD[s][a],2)) for ns in empty_MDP_bkp.tD[s][a]]
#         print("\t", ns_tuples)

In [80]:
# for s_i in range(50):
#     for a_i in range(2):
#         print(empty_MDP_bkp.tranidxMatrix_cpu[a_i,s_i],empty_MDP.tranidxMatrix_cpu[a_i,s_i])
#         print()        

In [81]:
empty_MDP_bkp.do_optimal_backup(mode = "GPU",n_backups=250)

In [82]:
empty_MDP_bkp.valueDict

{'unknown_state': 0.0,
 'end_state': 0.0,
 (0.013242853, 0.027749438, -0.010012642, 0.03907717): 97.0296401977539,
 (0.013797842, 0.22301352, -0.009231098, -0.25674796): 96.99964141845703,
 (0.018258113, 0.41826606, -0.014366058, -0.55232817): 96.9693374633789,
 (0.026623433, 0.22334877, -0.025412621, -0.26420587): 96.93872833251953,
 (0.031090409, 0.028598582, -0.030696739, 0.020354565): 96.9078140258789,
 (0.031662382, 0.224147, -0.030289646, -0.2818533): 96.87657928466797,
 (0.036145322, 0.41968763, -0.035926715, -0.5839335): 96.84503173828125,
 (0.04453907, 0.22508688, -0.047605384, -0.30278093): 96.81316375732422,
 (0.04904081, 0.42085385, -0.053661, -0.61008936): 96.78097534179688,
 (0.057457887, 0.22652142, -0.06586279, -0.33477908): 96.74846649169922,
 (0.061988316, 0.4225159, -0.07255837, -0.64748305): 96.71562194824219,
 (0.07043863, 0.61856973, -0.08550803, -0.9621035): 96.68244934082031,
 (0.08281003, 0.42469448, -0.1047501, -0.69746184): 96.64894104003906,
 (0.091303915, 0

In [57]:
empty_MDP.tranProbMatrix_cpu[:,5]

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.39795917, 0.33673468, 0.15306123, 0.08163265, 0.03061225]],
      dtype=float32)

In [26]:
empty_MDP.vD_cpu[5]

array([1.], dtype=float32)

In [27]:
empty_MDP.vD_gpu[5]

array([1.], dtype=float32)

In [10]:
empty_MDP_bkp.tranProbMatrix_cpu[:,5]

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.25510204, 0.24489796, 0.2244898 , 0.1734694 , 0.10204082]],
      dtype=float32)

In [11]:
empty_MDP_bkp.tranidxMatrix_cpu = empty_MDP.tranidxMatrix_cpu 
empty_MDP_bkp.tranProbMatrix_cpu = empty_MDP.tranProbMatrix_cpu 
empty_MDP_bkp.rewardCountMatrix_cpu = empty_MDP.rewardCountMatrix_cpu 
empty_MDP_bkp.rewardMatrix_cpu = empty_MDP.rewardMatrix_cpu 

In [12]:
# empty_MDP_bkp.refresh_cache_dicts()

In [13]:
def refresh_cache_dicts(self):
    self.tD = defaultdict(init2zero_def_def_dict)  # Transition Probabilities
    self.rD = defaultdict(init2zero_def_def_dict)  # Transition Probabilities

    for s, s_i in self.s2i.items():
        for a in self.A:
            for ns_slot, ns_i in enumerate(self.tranidxMatrix_cpu[self.a2i[a], s_i]):
                ns,a_i = self.i2s[ns_i], self.a2i[a]
                if self.tranProbMatrix_cpu[a_i, s_i, ns_slot] >0:
                    print(f"ns:{ns},ns_i:{ns_i}, a_i:{a_i}, a:{a}, s:{s}, s_i:{s_i} ,  ns_slot:{ns_slot}")
                    self.tD[s][a][ns] = self.tranProbMatrix_cpu[a_i, s_i, ns_slot]
                    self.rD[s][a][ns] = self.rewardMatrix_cpu[a_i, s_i, ns_slot]
                # Transition Probabilities

In [14]:
empty_MDP.rewardMatrix_cpu[:,5]

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.65722173, 0.6358138 , 0.6016565 , 0.48741028, 0.17818214]],
      dtype=float32)

In [21]:
empty_MDP.tranProbMatrix_cpu[:,5]

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.25510204, 0.24489796, 0.2244898 , 0.1734694 , 0.10204082]],
      dtype=float32)

In [22]:
empty_MDP.tranProbMatrix_gpu[:,5]

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.25510204, 0.24489796, 0.2244898 , 0.1734694 , 0.10204082]],
      dtype=float32)

In [23]:
empty_MDP.tranidxMatrix_gpu[:,5]

array([[ 6,  4,  8, 10, 14],
       [ 5,  7,  9, 13, 12]], dtype=uint64)

In [24]:
empty_MDP.tranidxMatrix_cpu[:,5]

array([[ 6,  4,  8, 10, 14],
       [ 5,  7,  9, 13, 12]], dtype=uint64)

In [15]:
empty_MDP.rewardMatrix_gpu[:,5]

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.65722173, 0.6358138 , 0.6016565 , 0.48741028, 0.17818214]],
      dtype=float32)

In [16]:
# empty_MDP.tranProbMatrix_gpu[:,5]

In [17]:
# refresh_cache_dicts(empty_MDP)
# empty_MDP.tranProbMatrix_cpu

In [19]:
# for s in empty_MDP.tD:
#     for a in empty_MDP.tD[s]:
#         print(empty_MDP.s2i[s],a)
#         ns_tuples = [(empty_MDP.s2i[ns],round(empty_MDP.tD[s][a][ns],2), round(empty_MDP.rD[s][a][ns],2)) for ns in empty_MDP.tD[s][a]]
#         print("\t", ns_tuples)
#         ns_tuples = [(empty_MDP_bkp.s2idx[ns],round(empty_MDP_bkp.tD[s][a][ns],2), round(empty_MDP_bkp.rD[s][a],2)) for ns in empty_MDP_bkp.tD[s][a]]
#         print("\t", ns_tuples)

In [20]:
myAgent.mdp_T.valueDict

{'unknown_state': 0.0,
 'end_state': 0.0,
 (-0.04859533, -0.024868723, 0.027136736, 0.022612793): 1.0,
 (-0.049092706, -0.22036912, 0.027588991, 0.32373264): 1.0,
 (-0.05350009, -0.41587284, 0.034063645, 0.62498677): 1.0,
 (-0.061817545, -0.22124256, 0.04656338, 0.34322348): 1.0,
 (-0.0662424, -0.41699496, 0.05342785, 0.6502185): 1.0,
 (-0.07458229, -0.22265631, 0.066432215, 0.3748268): 1.0,
 (-0.07903542, -0.4186559, 0.07392876, 0.6876944): 1.0,
 (-0.08740854, -0.2246337, 0.08768264, 0.4191713): 1.0,
 (-0.09190121, -0.4208816, 0.096066065, 0.7381579): 1.0,
 (-0.10031885, -0.61718965, 0.11082923, 1.0594623): 1.0,
 (-0.112662636, -0.42369625, 0.13201848, 0.8035201): 1.0,
 (-0.12113656, -0.23060761, 0.14808887, 0.5551082): 1.0,
 (-0.12574871, -0.42746451, 0.15919104, 0.89054084): 1.0,
 (-0.13429801, -0.6243465, 0.17700185, 1.2287344): 1.0,
 (-0.14678493, -0.43188757, 0.20157655, 0.9963237): 0.724399209022522}

In [8]:
myAgent.mdp_T.pD_cpu = np.zeros((myAgent.mdp_T.MAX_S,)).astype('uint')  # Optimal Policy Vector
myAgent.mdp_T.vD_cpu = np.zeros((myAgent.mdp_T.MAX_S, 1)).astype('float32')
myAgent.mdp_T.qD_cpu = np.zeros((myAgent.mdp_T.MAX_S, len(myAgent.mdp_T.A))).astype('float32')

In [9]:

def opt_bellman_backup_step_cpu(self):
    backup_error = 0
    for s, s_i in self.s2i.items():
        for a, a_i in self.a2i.items():
            expected_ns_val = np.sum(self.tranProbMatrix_cpu[a_i,s_i] *
                            np.array([self.vD_cpu[ns_i] for ns_i in self.tranidxMatrix_cpu[a_i,s_i]]).squeeze())
            expected_reward = np.sum(self.tranProbMatrix_cpu[a_i,s_i] * self.rewardMatrix_cpu[a_i,s_i])
            self.qD_cpu[s_i,a_i] = expected_reward + self.vi_params["gamma"] * expected_ns_val

        new_val = np.max(self.qD_cpu[s_i])

        backup_error = max(backup_error, abs(new_val - self.vD_cpu[s_i]))
        self.vD_cpu[s_i] = new_val
        self.pD_cpu[s_i] = np.argmax(self.qD_cpu[s_i])

    self.curr_vi_error = backup_error
    return backup_error

In [10]:
for i in range(500):
    opt_bellman_backup_step_cpu(myAgent.mdp_T)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [157]:
myAgent_bkp.mdp_T.vD_cpu.shape,myAgent.mdp_T.vD_cpu.shape

((1200, 1), (1200, 1))

In [71]:
# myAgent_bkp.mdp_T.valueDict

In [32]:
myAgent.mdp_T.curr_vi_error

0.0

In [28]:
myAgent_bkp.mdp_T.valueDict

{'unknown_state': 0.0,
 'end_state': 0.0,
 (-0.031631507, -0.042020194, 0.04810162, 0.03468045): 92.48587036132812,
 (-0.03247191, -0.23779775, 0.048795227, 0.34214354): 92.41046905517578,
 (-0.037227865, -0.43357873, 0.055638097, 0.6498056): 92.33431243896484,
 (-0.04589944, -0.62942976, 0.06863421, 0.95947665): 92.25736999511719,
 (-0.058488034, -0.82540387, 0.08782374, 1.2729084): 92.1796646118164,
 (-0.074996114, -0.6315055, 0.11328191, 1.0089687): 92.10116577148438,
 (-0.087626226, -0.82794225, 0.13346128, 1.334969): 92.02188110351562,
 (-0.10418507, -0.6347307, 0.16016066, 1.08685): 91.94178771972656,
 (-0.11687968, -0.83156043, 0.18189767, 1.4252023): 91.8573989868164,
 (-0.043882217, 0.03853046, 0.008046919, -0.048267525): 93.03724670410156,
 (-0.04311161, -0.15670595, 0.007081568, 0.24694337): 92.9674072265625,
 (-0.046245728, -0.35192832, 0.012020435, 0.5418515): 92.89685821533203,
 (-0.053284295, -0.15697736, 0.022857467, 0.25298017): 92.82561492919922,
 (-0.056423843, -0.35

In [27]:
myAgent.smoothing = True
myAgent.soft_q = False
myAgent.mdp_T.plcy_lift_k = 11
myAgent.mdp_T.smooth_with_seen = True

sum_reward_running = {policy_name:evaluate_on_env(env,  
                                                  policy, 
                                                  eps_count=50,progress_bar=True)[0]
                      for policy_name,policy in myAgent.policies.items()}
print(sum_reward_running)

100%|██████████| 50/50 [00:04<00:00, 12.07it/s]

{'optimal': 200.0, 'random': 19.3, 'eps_optimal': 200.0, 'safe': 200.0}


In [26]:
# myAgent.smoothing = True
# myAgent.soft_q = False
# myAgent.mdp_T.plcy_lift_k = 11
# myAgent.mdp_T.smooth_with_seen = True

# sum_reward_running = {policy_name:evaluate_on_env(env,  
#                                                   policy, 
#                                                   eps_count=50,progress_bar=True)[0]
#                       for policy_name,policy in myAgent_bkp.policies.items()}
# print(sum_reward_running)

100%|██████████| 50/50 [00:01<00:00, 38.35it/s]

{'optimal': 196.58, 'random': 18.78, 'eps_optimal': 186.32, 'safe': 200.0}


In [23]:
evaluate_on_env(env,  myAgent.policies["optimal"], eps_count=50,progress_bar=True)[0]


100%|██████████| 50/50 [00:03<00:00, 13.39it/s]


200.0

In [25]:
evaluate_on_env(env,  myAgent_bkp.policies["optimal"], eps_count=50,progress_bar=True)[0]

100%|██████████| 50/50 [00:01<00:00, 37.79it/s]


195.02

In [ ]:
for s in myAgent.mdp_cache:
    for a in myAgent.mdp_cache[s]:
        for ns in myAgent.mdp_cache[s][a]:

In [17]:
a = {1:2, 3:1}
max(a, key=a.get)

1

In [10]:
from pycuda import compiler, gpuarray
from collections import defaultdict
import time
import math as mth
from bigmdp.mdp.kernels import complex_vi_kernel_code_template as vi_kernel_template
import numpy as np
from tqdm import tqdm
from copy import deepcopy as cpy
from collections import deque
import random
from sklearn.neighbors import KDTree
from collections import Counter

In [11]:
e_vD_gpu = gpuarray.to_gpu(np.zeros((100, 1)).astype('float32'))
e_vD_cpu = np.zeros((100, 1)).astype('float32')

In [12]:
for i in tqdm(range(int(1e6))):
    e_vD_cpu[1][0] = 1

100%|██████████| 1000000/1000000 [00:00<00:00, 1442770.47it/s]


In [13]:
for i in tqdm(range(int(1e6))):
    e_vD_gpu[1][0] = 1

  0%|          | 0/1000000 [00:00<?, ?it/s]


TypeError: src must be GPUArray or ndarray

In [36]:
tranCountMatrix_cpu = np.ones((2, 4, 3)).astype('float32')
tranProbMatrix_cpu = np.zeros((2, 4, 3)).astype('float32')
tranCountMatrix_cpu[0,0,-1] = 0

In [37]:
a_i = 0
s_i = 0
tranProbMatrix_cpu[a_i,s_i] = tranCountMatrix_cpu[a_i,s_i]/np.sum(tranCountMatrix_cpu[a_i,s_i])

In [38]:
tranProbMatrix_cpu

array([[[0.5, 0.5, 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]],

       [[0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ],
        [0. , 0. , 0. ]]], dtype=float32)

In [31]:
tranCountMatrix_cpu

array([[[5., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[5., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)